
# 🚀 Azure AI Foundry: Foundational 101 Set-Up

This notebook serves as a comprehensive guide to getting started with **Azure AI Foundry**. It will walk you through the essential setup, project creation, and usage of core features like model inference, prompt templates, and Azure AI Search.

> **Note:** This notebook assumes you have access to the Azure AI Foundry SDK, the required Azure resources, and a valid project connection string.

## 🧰 **Prerequisites**
To ensure you can follow along, please make sure you have the following in place:
- **Azure Subscription** with permissions to create resources.
- **Azure CLI** installed to authenticate and access resources.
- **Python 3.8+** installed on your system.
- **AI Foundry Project** created via the Azure portal. [official Azure AI Foundry documentation](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio).

## 📚 **Table of Contents**
1. [Project Setup](#1-project-setup)
2. [Model Inference](#2-model-inference)
3. [Prompt Templates](#3-prompt-templates)
4. [Observability & Monitoring](#4-observability-monitoring)
5. [Evaluations](#5-evaluations)

## 📁 1. Project Setup

In [1]:
import os
from datetime import datetime
from pprint import pprint
from dotenv import load_dotenv

# Define the target directory (change yours)
TARGET_DIRECTORY = r"C:\Users\pablosal\Desktop\gbb-ai-llm-slm-evaluation-framework"

# Check if the directory exists
if os.path.exists(TARGET_DIRECTORY):
    # Change the current working directory
    os.chdir(TARGET_DIRECTORY)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {TARGET_DIRECTORY} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbb-ai-llm-slm-evaluation-framework


In [4]:
# Step 1: Import necessary libraries
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Step 2: Set your connection string (replace with your actual connection string)
project_connection_string = os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING")

# Step 3: Create the project client
project = AIProjectClient.from_connection_string(
    conn_str=project_connection_string,
    credential=DefaultAzureCredential()
)

# Verify connection
print("Project client created successfully.")
    

Project client created successfully.


## 🤖 2. Model Inference

In [4]:
from azure.ai.inference.tracing import AIInferenceInstrumentor

# Initialize the AIInferenceInstrumentor for tracing
AIInferenceInstrumentor().instrument()

# Get the chat completions client
chat = project.inference.get_chat_completions_client()

# Send a request to the model and get the response
response = chat.complete(
    model="Phi-3.5-mini-instruct",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant"},
        {"role": "user", "content": "Can you explain the theory of relativity in simple terms?"},
    ]
)

# Print the response from the AI model
print(f"🤖 AI Response: {response.choices[0].message.content}")

🤖 AI Response:  The theory of relativity, proposed by Albert Einstein, encompasses two theories: special relativity and general relativity.


Special relativity, introduced in 1905, deals with the physics of motion in the absence of gravity and holds that the laws of physics are the same for all non-accelerating observers. It introduced the idea that time and space are relative and not absolute concepts. This means that time can slow down or speed up depending on how fast one is moving relative to something else. It also shows that mass and energy are equivalent, as expressed in the famous equation E=mc^2, where E is energy, m is mass, and c is the speed of light.


General relativity, published in 1915, extends the theory to include gravity. It describes gravity not as a force, but as a curvature of spacetime caused by mass and energy. Massive objects like stars and planets curve the spacetime around them, and other objects move along these curves, which we perceive as gravitational a

## 📄 3. Prompt Templates

In [5]:
# Step 1: Import the PromptTemplate class
from azure.ai.inference.prompts import PromptTemplate

# Step 2: Create a prompt template
prompt_template = PromptTemplate.from_string("""
    system: You are a helpful AI assistant.
    user: Write a haiku about the beauty of technology.
""")

# Step 3: Generate system message from the template
messages = prompt_template.create_messages()

# Step 4: Print the generated messages
print(messages)
    

[{'role': 'system', 'content': 'system: You are a helpful AI assistant.\nuser: Write a haiku about the beauty of technology.'}]


## 4. 📡 Tracing and Observability

Tracing and observability are essential for monitoring the performance, latency, and errors of AI applications. By enabling OpenTelemetry tracing in Azure AI Foundry, you can visualize and analyze the performance of API requests and model inferences. This is useful for identifying slow responses, debugging errors, and optimizing system performance.

With Azure Monitor and Application Insights, you can log telemetry, track request flows, and observe traces in near real-time.

In [6]:
## set-up opentelemetry
# install !pip install azure-monitor-opentelemetry azure-ai-inference[opentelemetry] opentelemetry-exporter-otlp
from azure.core.settings import settings

# Set tracing implementation to OpenTelemetry
settings.tracing_implementation = "opentelemetry"

from azure.ai.inference.tracing import AIInferenceInstrumentor

# Instrument AI Inference API to enable tracing
AIInferenceInstrumentor().instrument()

from azure.monitor.opentelemetry import configure_azure_monitor

# Retrieve the Application Insights connection string from your AI project
application_insights_connection_string = project.telemetry.get_connection_string()

# Enable Azure Monitor tracing if the connection string exists
if application_insights_connection_string:
    configure_azure_monitor(connection_string=application_insights_connection_string)
else:
    print("❌ Application Insights is not enabled for this project.")


In [7]:
# Step 1: Import necessary libraries
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
from azure.core.settings import settings 
from azure.ai.inference.tracing import AIInferenceInstrumentor 
from azure.ai.inference.prompts import PromptTemplate

# Step 2: Set tracing implementation to OpenTelemetry
settings.tracing_implementation = "opentelemetry"

# Step 3: Instrument AI Inference API
AIInferenceInstrumentor().instrument()

# Step 4: Get a chat inferencing client using the project's default model inferencing endpoint
chat_client = project.inference.get_chat_completions_client()

# Step 5: Enable instrumentation of AI packages (inference, agents, openai, langchain)
project.telemetry.enable()

# Step 6: Log traces to the project's application insights resource
application_insights_connection_string = project.telemetry.get_connection_string()
if application_insights_connection_string:
    configure_azure_monitor(connection_string=application_insights_connection_string)

# Step 7: Create a tracer
tracer = trace.get_tracer(__name__)

# Step 8: Create the prompt template
prompt_template = PromptTemplate.from_string("""
    system: You are a helpful assistant for {{user_name}}.
    user: {{user_message}}
""")

# Step 9: Define user name for the template
user_name = "Alex"

# Step 10: Start a multi-turn conversation with tracing
with tracer.start_as_current_span("multi_turn_chat_example"):
    # Turn 1
    messages = prompt_template.create_messages(user_name=user_name, user_message="Can you explain the theory of relativity in simple terms?")
    response_1 = chat_client.complete(model="Phi-3.5-mini-instruct", messages=messages)
    print(f"🤖 AI Response 1: {response_1.choices[0].message.content}")
    
    # Turn 2
    messages = prompt_template.create_messages(user_name=user_name, user_message="Can you provide a simple analogy to understand it better?")
    response_2 = chat_client.complete(model="Phi-3.5-mini-instruct", messages=messages)
    print(f"🤖 AI Response 2: {response_2.choices[0].message.content}")
    
    # Turn 3
    messages = prompt_template.create_messages(user_name=user_name, user_message="How is it related to gravity and time?")
    response_3 = chat_client.complete(model="Phi-3.5-mini-instruct", messages=messages)
    print(f"🤖 AI Response 3: {response_3.choices[0].message.content}")


🤖 AI Response 1:  Absolutely, Alex! The theory of relativity is actually made up of two related theories proposed by Albert Einstein in the early 20th century. Here's a simplified explanation:

1. Special Relativity (1905):
This part of the theory deals with objects moving at a constant speed in a straight line, particularly when they're moving close to the speed of light. Here are the key points:

   a. Time can pass differently for people depending on their speed. Imagine two friends, one staying on Earth and the other traveling at a speed close to the speed of light. When they reunite, the traveler might have aged less than the friend on Earth. This is known as time dilation.

   b. Space and time are linked together into a single thing called spacetime. Imagine a trampoline with a heavy ball (like Earth) in the middle causing a dip. Now, if you roll smaller balls around the dip, they'll follow curved paths. This is how gravity works in General Relativity.

2. General Relativity (19

## 🛡️ 5. Evaluations

#### 2️⃣ Local Evaluation


In [16]:
#!pip install azure-ai-evaluation

In [8]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create AIProjectClient from environment variables
project = AIProjectClient.from_connection_string(
    conn_str=os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING"),
    credential=DefaultAzureCredential()
)

In [9]:
from azure.ai.evaluation import GroundednessEvaluator, ViolenceEvaluator, RelevanceEvaluator

# Configure model for the evaluators
model_config = {
    "azure_endpoint": os.getenv("AZURE_AOAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_AOAI_API_KEY"),
    "azure_deployment": os.getenv("AZURE_AOAI_COMPLETION_MODEL_DEPLOYMENT_ID"),
    "api_version": os.getenv("AZURE_AOAI_DEPLOYMENT_VERSION"),
}

# Initialize Evaluators
groundedness_eval = GroundednessEvaluator(model_config)
violence_eval = ViolenceEvaluator(credential=DefaultAzureCredential(), azure_ai_project=project.scope)
relevance_eval = RelevanceEvaluator(model_config)


In [6]:
from azure.ai.evaluation import ContentSafetyEvaluator

ContentSafetyEvaluator?

Init signature: ContentSafetyEvaluator(credential, azure_ai_project, **kwargs)
Docstring:     
.. note::    This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Initialize a content safety evaluator configured to evaluate content safetry metrics for QA scenario.

:param credential: The credential for connecting to Azure AI project. Required
:type credential: ~azure.core.credentials.TokenCredential
:param azure_ai_project: The scope of the Azure AI project.
    It contains subscription id, resource group, and project name.
:type azure_ai_project: ~azure.ai.evaluation.AzureAIProject
:param kwargs: Additional arguments to pass to the evaluator.
:type kwargs: Any
:return: A function that evaluates content-safety metrics for "question-answering" scenario.

.. admonition:: Example:

    .. literalinclude:: ../samples/evaluation_samples_evaluate.py
        :start-after: [START content_safety_evaluator]
        :end-be

In [10]:
import json

directory = "my_utils/evals/data"
file_name = "evaluation_data.jsonl"

os.makedirs(directory, exist_ok=True)

content = [
    {"query": "What is the capital of France?", "context": "France is in Europe", "response": "Paris is the capital of France.", "ground_truth": "Paris"},
    {"query": "Who discovered gravity?", "context": "Isaac Newton discovered gravity when an apple fell on his head.", "response": "Isaac Newton.", "ground_truth": "Isaac Newton"}
]

# Save the content to the file
with open(os.path.join(directory, file_name), 'w') as file:
    for item in content:
        file.write(f"{json.dumps(item)}\n")

In [13]:
import json
from azure.ai.evaluation import evaluate

# Run the evaluation locally
try:
    results = evaluate(
        data="./my_utils/evals/data/evaluation_data.jsonl",
        evaluators={
            "groundedness": groundedness_eval,
            "relevance": relevance_eval
        },
        evaluator_config={
            "default": {
                "column_mapping": {
                    "query": "${data.query}",
                    "context": "${data.context}",
                    "response": "${data.response}"
                }
            }
        },
        output_path="my_utils/evals/results/local_evaluation_results.json"
    )

except Exception as e:
    print(f"An error occurred during evaluation: {e}")

[2024-12-09 13:21:08 -0600][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-09 13:21:08 -0600][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-09 13:21:08 -0600][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_zo0n23x8_20241209_132107_859762, log path: C:\Users\pablosal\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_zo0n23x8_20241209_132107_859762\logs.txt
[2024-12-09 13:21:08 -0600][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_wvit_90g_20241209_132107_859762, log path: C:\Users\pablosal\.promptflow\.runs\azure_ai_eva

Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_wvit_90g_20241209_132107_859762
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_zo0n23x8_20241209_132107_859762


In [14]:
results

{'rows': [{'inputs.query': 'What is the capital of France?',
   'inputs.context': 'France is in Europe',
   'inputs.response': 'Paris is the capital of France.',
   'inputs.ground_truth': 'Paris',
   'outputs.groundedness.groundedness': 3,
   'outputs.groundedness.gpt_groundedness': 3,
   'outputs.groundedness.groundedness_reason': 'The RESPONSE is accurate but introduces additional information not supported by the CONTEXT.',
   'outputs.relevance.relevance': 4,
   'outputs.relevance.gpt_relevance': 4,
   'outputs.relevance.relevance_reason': 'The response is accurate and complete, directly answering the question without any extraneous information.'},
  {'inputs.query': 'Who discovered gravity?',
   'inputs.context': 'Isaac Newton discovered gravity when an apple fell on his head.',
   'inputs.response': 'Isaac Newton.',
   'inputs.ground_truth': 'Isaac Newton',
   'outputs.groundedness.groundedness': 4,
   'outputs.groundedness.gpt_groundedness': 4,
   'outputs.groundedness.groundedne

#### Send to the Cloud (AI Foundry Evaluations)

In [22]:
azure_ai_project = {
                    "subscription_id": AZURE_AI_STUDIO_SUBSCRIPTION_ID,
                    "resource_group_name": AZURE_AI_STUDIO_RESOURCE_GROUP_NAME,
                    "project_name": AZURE_AI_STUDIO_PROJECT_NAME
                }

In [23]:
import json
from azure.ai.evaluation import evaluate

try:
    results = evaluate(
        data="./my_utils/evals/data/evaluation_data.jsonl",
        evaluators={
            "groundedness": groundedness_eval,
            "relevance": relevance_eval
        },
        evaluator_config={
            "default": {
                "column_mapping": {
                    "query": "${data.query}",
                    "context": "${data.context}",
                    "response": "${data.response}"
                }
            }
        },
        output_path="my_utils/evals/results/local_evaluation_results.json",
        azure_ai_project=azure_ai_project
    )
except Exception as e:
    print(f"An error occurred during evaluation: {e}")

[2024-12-09 13:31:00 -0600][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-09 13:31:00 -0600][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-09 13:31:00 -0600][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_z_gww5q0_20241209_133100_023603, log path: C:\Users\pablosal\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_z_gww5q0_20241209_133100_023603\logs.txt
[2024-12-09 13:31:00 -0600][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_9jyrobxo_20241209_133100_028825, log path: C:\Users\pablosal\.promptflow\.runs\azure_ai_eva

Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_z_gww5q0_20241209_133100_023603
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_9jyrobxo_20241209_133100_028825
2024-12-09 13:31:05 -0600   26948 execution.bulk     INFO     Finished 2 / 2 lines.
2024-12-09 13:31:05 -0600   26948 execution.bulk     INFO     Average execution time for completed lines: 2.73 seconds. Estimated time for incomplete lines: 0.0 seconds.


In [26]:
results['studio_url']

'https://ai.azure.com/build/evaluation/32881dd3-afda-4ad8-a6da-540ad9425534?wsid=/subscriptions/20c54e6d-27f8-4aeb-8127-3dfa557b9721/resourceGroups/dev-rg-ai-factory/providers/Microsoft.MachineLearningServices/workspaces/dev-factory'

#### 3️⃣ Remote (Cloud) Evaluation


In [ ]:
!pip install azure-identity azure-ai-projects azure-ai-ml

In [61]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import Evaluation, Dataset, EvaluatorConfiguration, ConnectionType
from azure.ai.evaluation import F1ScoreEvaluator, RelevanceEvaluator, ViolenceEvaluator

In [91]:
# Upload local file to Azure AI project
data_id, _ = project.upload_file("./my_utils/evals/data/evaluation_data.jsonl")
print(f"Uploaded local file with ID: {data_id}")

In [92]:
from azure.ai.evaluation import F1ScoreEvaluator, RelevanceEvaluator, ViolenceEvaluator
print("F1 Score evaluator id:", F1ScoreEvaluator.id)

In [68]:
default_connection = project.connections.get_default(connection_type=ConnectionType.AZURE_OPEN_AI)
model_config = {
    "azure_endpoint": os.getenv("AZURE_AOAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_AOAI_API_KEY"),
    "azure_deployment": os.getenv("AZURE_AOAI_COMPLETION_MODEL_DEPLOYMENT_ID"),
    "api_version": os.getenv("AZURE_AOAI_DEPLOYMENT_VERSION"),
}
model_config = default_connection.to_evaluator_model_config(deployment_name=model_config['azure_deployment'], api_version=model_config["api_version"])


In [70]:
model_config

{'azure_deployment': 'gpt-4o',
 'azure_endpoint': 'https://azureaistudiof7244419895.openai.azure.com',
 'type': 'azure_openai',
 'api_version': '2024-08-01-preview',
 'api_key': '/subscriptions/20c54e6d-27f8-4aeb-8127-3dfa557b9721/resourceGroups/dev-rg-ai-factory/providers/Microsoft.MachineLearningServices/workspaces/dev-factory/connections/azureaistudiof7244419895_aoai/credentials/key'}

In [94]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import Evaluation, Dataset, EvaluatorConfiguration
from azure.ai.evaluation import F1ScoreEvaluator

# Setup connection
connection_string = os.getenv("AZURE_AI_FOUNDRY_CONNECTION_STRING")
if not connection_string:
    raise ValueError("AZURE_AI_FOUNDRY_CONNECTION_STRING is missing or not set.")

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string
)

# Create the evaluation
evaluation = Evaluation(
    display_name="Cloud evaluation",
    description="Evaluation of dataset",
    data=Dataset(id=data_id),
    evaluators={
        "f1_score": EvaluatorConfiguration(
            id="azureml://registries/azureml/models/F1Score-Evaluator/versions/3"
        )
    }
)

In [96]:
try:
    response = project_client.evaluations.create(
        evaluation=evaluation,
        headers={
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
    )
    print("Evaluation submitted successfully!")
    print(f"Evaluation ID: {response.id}")
    print(f"Check status at: {response.properties.get('AiStudioEvaluationUri')}")
    
except Exception as e:
    if hasattr(e, 'response') and hasattr(e.response, 'content'):
        print("Binary content detected, saving to binary_response_output.bin")
        with open('binary_response_output.bin', 'wb') as f:
            f.write(e.response.content)
    else:
        print("An unexpected error occurred:", e)


For further information, explore the [official Azure AI Foundry documentation](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio).

Happy Coding! 🚀
    